# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as sqal
from sklearn.model_selection import train_test_split

In [2]:
# in_columns = 'message'
# cat_col_names = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [3]:
# load data from database
engine = sqal.create_engine('sqlite:///../data/DisasterResponse.db')
# inspector = sqal.inspect(engine)
# inspector.get_schema_names()  # -> ['main']
# inspector.get_table_names(schema='main')  # -> ['Message']
# inspector.get_columns('Message', schema='main')   # -> the goods

df = pd.read_sql_table('MessageCategorization', engine)

In [99]:
# correct outlying values in `related` column
df['related'] = np.clip(df['related'], 0, 1)

In [101]:
df = df.drop('child_alone', axis=1)

In [102]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [103]:
in_columns = 'message'
out_columns = list(df.columns)[4:]

In [104]:
text = df[in_columns].values
y = df[out_columns].values

In [105]:
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.33, random_state=42)

In [106]:
y_train

array([[1, 0, 0, ..., 0, 1, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [107]:
len(out_columns)

35

### 2. Write a tokenization function to process your text data

In [9]:
import spacy
en_nlp = spacy.load('en')

In [10]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [11]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [12]:
# https://realpython.com/natural-language-processing-spacy-python/ was helpful here
def tokenize(text):
    doc = en_nlp(text)
    lemmas = [token.lemma_ for token in doc if token not in stopwords and not token.is_punct]
    stems = [stemmer.stem(lemma) for lemma in lemmas]
    return stems

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import make_pipeline

In [108]:
# model = make_pipeline(TfidfVectorizer(tokenizer=tokenize, min_df=5), 
#                      MultiOutputClassifier(estimator=
#                                            LinearSVC(C=1.0, dual=False, multi_class='ovr', fit_intercept=True, 
#                                                      max_iter=100)))

### 3b. Split pipeline into stages

In [115]:
vect = TfidfVectorizer(tokenizer=tokenize, min_df=5)

In [116]:
clf = MultiOutputClassifier(estimator=
                            LinearSVC(C=1.0, dual=False, multi_class='ovr', fit_intercept=True, 
                                      max_iter=100))

In [117]:
model = make_pipeline(vect, clf)

In [118]:
model.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(min_df=5,
                                 tokenizer=<function tokenize at 0x12b374700>)),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LinearSVC(dual=False,
                                                           max_iter=100)))])>

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [82]:
%%time
pipe.fit(text_train, y_train)

CPU times: user 3min 22s, sys: 2.66 s, total: 3min 25s
Wall time: 3min 28s


Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(min_df=5,
                                 tokenizer=<function tokenize at 0x12b374700>)),
                ('multioutputclassifier',
                 MultiOutputClassifier(estimator=LinearSVC(dual=False,
                                                           max_iter=100)))])

In [83]:
# compute rigged score
pipe.score(text_train, y_train)

0.5131519016169438

In [84]:
# compute real score
pipe.score(text_test, y_test)

0.29079981507165975

Crud.  It's overfit.  Is there a way to weaken the LinearSVC model?

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [85]:
from sklearn.metrics import classification_report

In [86]:
%%time
y_pred = pipe.predict(text_test)

CPU times: user 1min 35s, sys: 1.39 s, total: 1min 36s
Wall time: 1min 43s


In [90]:
len(out_columns)

36

In [97]:
print(classification_report(y_test, y_pred, target_names=out_columns, zero_division=1))

                        precision    recall  f1-score   support

               related       0.86      0.90      0.88      6598
               request       0.78      0.60      0.68      1472
                 offer       1.00      0.00      0.00        38
           aid_related       0.73      0.68      0.70      3545
          medical_help       0.57      0.28      0.38       701
      medical_products       0.67      0.28      0.39       446
     search_and_rescue       0.76      0.14      0.23       226
              security       0.50      0.02      0.04       160
              military       0.61      0.31      0.41       267
                 water       0.74      0.57      0.64       543
                  food       0.82      0.68      0.74       965
               shelter       0.75      0.58      0.65       775
              clothing       0.67      0.29      0.41       127
                 money       0.53      0.26      0.35       191
        missing_people       0.79      

In [ ]:
y_test.shape

In [ ]:
for i in range(y_test.shape[1]):
    print(cat_col_names[i])
    print(classification_report(y_test[:,i], y_pred[:,i], zero_division=1))

https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics

In [75]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def genTable(true, pred, zero_division='warn'):
    table = np.empty(shape=(true.shape[1], 3))
    for i in range(true.shape[1]):
        table[i] = [precision_score(true[:,i], pred[:,i], zero_division=zero_division), \
                      recall_score(true[:,i], pred[:,i], zero_division=zero_division), \
                      f1_score(true[:,i], pred[:,i], zero_division=zero_division)]
    return table

def printScores(labels, true, pred, zero_division='warn'):
    results = genTable(true, pred, zero_division=zero_division)
    print("{0:>20}".format(''), 'prec', 'recall', 'f1', sep='\t')
    for i in range(results.shape[0]):
        print("{0:>20}".format(labels[i]), '%.2f' % results[i, 0], '%.2f' % results[i,1], '%.2f' % results[i,2], sep = '\t')

In [77]:
#printScores(out_columns, y_test, y_pred, zero_division=0)

### 6. Improve your model

####  Let's split the `Vectorizer` out of the pipeline, precompute it on the training and test sets, and write the results to disk. 
Parameter searching is a slow business and we want to speed things up as much as we can.

In [ ]:
# vect = TfidfVectorizer(tokenizer=tokenize, min_df=5)

In [ ]:
# %%time
# # Pre-calculate X_train and X_test for testing the categorization pipeline - takes 5 minutes to calculate
# X_train1 = vect.fit_transform(text_train)
# X_test1 = vect.transform(text_test)

In [ ]:
# X_train1.shape

In [18]:
# https://stackoverflow.com/questions/52444921/save-numpy-array-using-pickle
import pickle as pkl

In [ ]:
# fileObject = open('severedPipelineXtrain.pkl', 'wb')
# pkl.dump(X_train1, fileObject)
# fileObject.close()

# fileObject = open('severedPipelineXtest.pkl', 'wb')
# pkl.dump(X_test1, fileObject)
# fileObject.close()

In [19]:
fileObject = open('severedPipelineXtrain.pkl', 'rb')
X_train = pkl.load(fileObject)
fileObject.close()

fileObject = open('severedPipelineXtest.pkl', 'rb')
X_test = pkl.load(fileObject)
fileObject.close()

In [20]:
type(X_train)

scipy.sparse.csr.csr_matrix

In [21]:
X_train.shape

(17564, 4859)

In [39]:
X_train[0,3826]

0.0

In [22]:
X_test.shape

(8652, 4859)

#### Alright.  Back to work.  Let's actually evaluate this classifier-only pipeline.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
                                           RandomForestClassifier(n_jobs=-1)))
#                                           AdaBoostClassifier()))
#                                           LogisticRegression(C=3, max_iter=30, warm_start=True, n_jobs=-1)))
#                                           KNeighborsClassifier(n_jobs=-1)))

LR doesn't work here because of the empty 'child_alone' feature.

In [ ]:
%%time
pipe.fit(X_train, y_train)

In [ ]:
print(pipe.score(X_test, y_test))
print(pipe.score(X_train, y_train))

At least it's not overfitting with `AdaBoostClassifier`, like it does with `RandomForestClassifier` (0.256, 0.995).

With KNeighbors we get (0.233, 0.264) but it took 16 minutes to compute the two scores; but less than a second to fit.

In [ ]:
%%time
y_pred = pipe.predict(X_test)

Took 5 mins with KNeighbors.  5 secs with RandomForest.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, target_names=out_columns, zero_division=1))

Wow!  RandomForest is a lot better than KNeighbors, but still awful.

In [ ]:
printScores(cat_col_names, y_test, y_pred, zero_division=0)

#### Let's do some parameter searching...

Use grid search to find better parameters. 

In [13]:
#from sklearn.linear_model import LogisticRegression
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.ensemble import AdaBoostClassifier
#from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

In [24]:
pipe = make_pipeline(MultiOutputClassifier(estimator=
                                            LinearSVC(C=1.0, dual=False, multi_class='ovr', fit_intercept=True, 
                                                      max_iter=100)))
#                                           RandomForestClassifier(n_jobs=-1)))  # This takes almost 11 minutes
#                                            AdaBoostClassifier(base_estimator=
#                                                               DecisionTreeClassifier(max_depth=1),
#                                                n_estimators=10, learning_rate=1))) # 1 minute
#                                           LogisticRegression(C=3, max_iter=30, warm_start=True))) 
#                                                              solver='saga', penalty='elasticnet')))
                                                              # class_weight='balanced')))

In [47]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('multioutputclassifier',
                 MultiOutputClassifier(estimator=LinearSVC(dual=False,
                                                           max_iter=100)))])

In [27]:
from sklearn.model_selection import GridSearchCV

In [55]:
# param_grid = {}
# param_grid = {'multioutputclassifier__estimator__n_estimators': [10, 30, 100]} 
# param_grid = {'multioutputclassifier__estimator__learning_rate': [0.3, 1.0, 3.0]}
# param_grid = {'multioutputclassifier__estimator__C': [3.0, 10.0, 30.0]}
# param_grid = {'multioutputclassifier__estimator__l1_ratio': [0.1, 0.3, 0.7, 1.0]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_split': [2, 4, 8, 16]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__min_samples_leaf': [1, 2, 4, 8]}
# param_grid = {'multioutputclassifier__estimator__base_estimator__max_depth': [1, 2, 4, 8]}
# param_grid = {'multioutputclassifier__estimator__C': [0.3, 1.0, 3.0, 10.0]}
param_grid = {'multioutputclassifier__estimator__C': [0.01, 0.03, 0.1, 0.3]}

In [56]:
grid = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5, n_jobs=-1)

In [57]:
%%time
grid.fit(X_train, y_train)

CPU times: user 6.03 s, sys: 126 ms, total: 6.16 s
Wall time: 32 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('multioutputclassifier',
                                        MultiOutputClassifier(estimator=LinearSVC(dual=False,
                                                                                  max_iter=100)))]),
             n_jobs=-1,
             param_grid={'multioutputclassifier__estimator__C': [0.01, 0.03,
                                                                 0.1, 0.3]})

With Ada(RandomForest) fit() ran 53 minutes before I stopped it.

25min for miniscule benefit - LR with saga not worth pursuing.  (Although the highest score fit (l1_ratio = 0.7) was only 1/3 the duration of the most expensive.  Could be because of the warm_start setting.)

For the DecisionTreeClassifier, the default value of `min_samples_split` of 2 is the best, but the effect is very weak. [2,4,8,16]

Equilvalently, for DTC, the default value of `min_samples_leaf` was best, and varying it made no difference. [1,2,4,8]

In [58]:
grid.best_params_

{'multioutputclassifier__estimator__C': 0.3}

In [59]:
grid.best_score_

0.30044484949167005

In [60]:
grid.score(X_test, y_test)

0.3036292186777624

In [61]:
grid.cv_results_

{'mean_fit_time': array([3.26463957, 4.26961055, 6.34450097, 7.92812238]),
 'std_fit_time': array([0.02928138, 0.25343928, 0.09593142, 0.68719226]),
 'mean_score_time': array([0.03549929, 0.05012698, 0.06139317, 0.0368516 ]),
 'std_score_time': array([0.00848184, 0.01286576, 0.01678532, 0.01692735]),
 'param_multioutputclassifier__estimator__C': masked_array(data=[0.01, 0.03, 0.1, 0.3],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'multioutputclassifier__estimator__C': 0.01},
  {'multioutputclassifier__estimator__C': 0.03},
  {'multioutputclassifier__estimator__C': 0.1},
  {'multioutputclassifier__estimator__C': 0.3}],
 'split0_test_score': array([0.21605465, 0.24793624, 0.27384002, 0.29461998]),
 'split1_test_score': array([0.18758896, 0.22658696, 0.28010248, 0.30429832]),
 'split2_test_score': array([0.20922289, 0.24366638, 0.27554796, 0.29177341]),
 'split3_test_score': array([0.20210646, 0.23939653, 0.27839453, 0.

DecisionTreeClassifier achieved (0.243, 0.238) with max_depth=2.  Grid search took 7 minutes.  Both min_samples_split and min_samples_leaf were negligable.

In [62]:
y_pred = grid.best_estimator_.predict(X_test)

In [68]:
y_pred = grid.predict(X_test)

In [ ]:
# DecisionTreeClassifier base_estimator
parameters = {'min_samples_leaf': [1], 'min_samples_split': [2], 'max_depth': [2]}

# AdaBoostClassifier estimator: I also tried varying 'n_estimators' and 'learning_rate'.
# This had minimal impact so I chose speed over optimization.

cv = 0.243



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [98]:
printScores(out_columns, y_test, y_pred, zero_division=0)

                    	prec	recall	f1
             related	0.86	0.90	0.88
             request	0.78	0.60	0.68
               offer	0.00	0.00	0.00
         aid_related	0.73	0.68	0.70
        medical_help	0.57	0.28	0.38
    medical_products	0.67	0.28	0.39
   search_and_rescue	0.76	0.14	0.23
            security	0.50	0.02	0.04
            military	0.61	0.31	0.41
               water	0.74	0.57	0.64
                food	0.82	0.68	0.74
             shelter	0.75	0.58	0.65
            clothing	0.67	0.29	0.41
               money	0.53	0.26	0.35
      missing_people	0.79	0.11	0.19
            refugees	0.65	0.24	0.35
               death	0.76	0.41	0.54
           other_aid	0.49	0.21	0.29
infrastructure_related	0.41	0.10	0.15
           transport	0.65	0.17	0.27
           buildings	0.70	0.37	0.48
         electricity	0.59	0.21	0.31
               tools	0.00	0.00	0.00
           hospitals	0.67	0.05	0.09
               shops	0.00	0.00	0.00
         aid_centers	0.25	0.02	0.03
other_infrastructure	0.39	

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### See above.  I've already tested many things including various algorithms.

### 9. Export your model as a pickle file

#### Reassemble the full model:

In [ ]:
pipe = make_pipeline(TfidfVectorizer(tokenizer=tokenize, min_df=5), 
                     MultiOutputClassifier(estimator=
                                           AdaBoostClassifier(base_estimator=
                                                              DecisionTreeClassifier(max_depth=2),
                                                                n_estimators=10, learning_rate=1)))

In [ ]:
%%time
pipe.fit(text_train, y_train)

And, finally, export the model.

In [ ]:
import pickle as pkl

In [ ]:
# Much niftier way of handling filehandles.
with open('AdaDecision.pkl', 'wb') as f:
    pkl.dump(pipe, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.